In [1]:
import os
from tqdm import tqdm
from fastai.text.all import *

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
torch.cuda.set_device('cuda:6')

In [3]:
df_train = pd.read_pickle('./data_new/pan_14e_cls/train_essays.pickle')
df_test01 = pd.read_pickle('./data_new/pan_14e_cls/test01_essays.pickle')
df_test02 = pd.read_pickle('./data_new/pan_14e_cls/test02_essays.pickle')

In [4]:
len(df_train), len(df_test01), len(df_test02)

(200, 100, 200)

In [5]:
# the LM part

In [6]:
# Getting unique documents
uniq_list = []
for dfi, df in enumerate( [df_train, df_test01, df_test02] ):
    for i in range(len(df)):
        for doc in df.iloc[i,1]:
            if( doc not in uniq_list ):
                uniq_list.append(doc)
                
        if( df.iloc[i,2] not in uniq_list ):
            uniq_list.append( df.iloc[i,2] )
len(uniq_list)

1119

In [7]:
uniq_list = pd.DataFrame(uniq_list)
uniq_list

,0
0,"English, my English! xcrlfx Assesing ones strengths and weaknesses in any situation is a hard task. xcrlfx Usually we are not so good at recognizing our strengths, instead we spend our time being critical of ourselves, thus we tend to present a much more detailed side of our flaws. xcrlfx I will try to give you a reasonable picture of my good and not quite as good sides, as I see them, in reference to the four skills that were mentioned in the guide-lines handed out earlier in regard of this essay. y biggest weakness by far is reading in English. xcrlfx I most of the time find it q..."
1,"Due to the increasing violence that we're daily exposed to, a need to take action has arisen. xcrlfx Television broadcasters and cable suppliers have been asked to endorse a computer-chip (the V-chip). xcrlfx This chip will enable parents to protect their children from ""the rude chocks of the tube"" (G. xcrlfx The V-chip will read encoded signals from the suppliers and on scale from one to four parents may determine what should be allowed on their television-sets and what shouldn't. xcrlfx Gina Bellafante, writes about the V-chip in TIME Magazine, July, 24, 1995 under the topic ""locking..."
2,"An peculiar lady crossed my path, once again! xcrlfx Reading David Malouf's ""Remembering Babylon"" you come across many different, yet similar characters and we get to know them in the way that Malouf provides us with their distinctive features or characteristics. xcrlfx I intend to concentrate my essay on the character of Mrs Hutchence, and provide you with statements, quotes and situations that in my opinion lead to characterise her as being peculiar in her ways, mysterious, big-hearted, practical and somehow respected by the people of Bowen. xcrlfx She's a very fascinating person th..."
3,"In the name of Religion, you can get away with anything! xcrlfx With this argumentative piece I've tried to shed a light on the consequences of religion. xcrlfx What goes on, apart from all of the good deeds man does in the name of religion. xcrlfx I've devided them in five headings, all regarding different aspects of the topic. xcrlfx I myself am not a believer. xcrlfx I'm a naturally born critic or sceptic, if you will. xcrlfx I firmly believe that it should be up to every man/woman to inividually process information and facts that are given or thrown upon us. xcrlfx Just becaus..."
4,English as a world language 1. xcrlfx Introduction My intention is to describe the importance of English and give a brief account of its history and how it has spread all over the world. xcrlfx I will begin with defining what a world language is and then I will describe the way English has reached the position it has today. xcrlfx I will continue with some up-to-date statistics and then mention some different varieties of English and explain the terms Pidgins and Creoles. xcrlfx I find it important to investigate how it is possible for a language spoken by so few at the beginning ...
...,...
1114,"In a country like Sweden the expectations of your knowledge in English are quite high. xcrlfx I, like almost everybody else here, have studied English since the age of ten. xcrlfx I have never had any serious problems to understand and to follow the English course in school. xcrlfx It hasn't been the most fascinating subject throughout the years, but much of that depends on the teacher, and how motivating he or she is. xcrlfx Somewhere along the way I must have changed my mind, since I am here now, continuing at university level. xcrlfx Besides English at school, the exposedness to En..."
1115,"Canadian Television In Canada there are a wider offer of TV programs and channels compared with Sweden. xcrlfx That is my experience from my stay in Canada. xcrlfx For me TV is entertainment and relaxation but also a way to enjoy my pastime. xcrlfx The most irritating thing about Canadian TV is the commercials. xcrlfx There are too many of them. xcrlfx For instance, if you watch a movie, it

In [8]:
# Make the LM DataLoader
dl = TextDataLoaders.from_df(uniq_list,
                            path='./data_new/pan_14e_cls/',
                            is_lm=True,
                            valid_pct=0.1,
                            text_col=0)
dl.show_batch(max_n=5)

/home/yifan/anaconda3/envs/avdv/lib/python3.9/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,text,text_
0,"xxbos xxmaj television and our conception of society xxmaj television has a powerful impact on many aspects of human life , both on individuals , family life and the society as a whole . xcrlfx xxmaj it provides for a substantial part of all information we receive , thus affecting our conception of the society and our own roles in it . xcrlfx xxmaj that is the topic for this piece of","xxmaj television and our conception of society xxmaj television has a powerful impact on many aspects of human life , both on individuals , family life and the society as a whole . xcrlfx xxmaj it provides for a substantial part of all information we receive , thus affecting our conception of the society and our own roles in it . xcrlfx xxmaj that is the topic for this piece of writing"
1,"them when he enters the xxunk , one xxunk statement could just as well have cost him his life . xcrlfx xxmaj fortunately , or unfortunately , that does not happen , but instead his speech brings about a civil war , just as he knew it would . \t▁ xcrlfx xxmaj marcus xxmaj antonius ' speech makes a very good example of the power of rhetoric , but it also illustrates","when he enters the xxunk , one xxunk statement could just as well have cost him his life . xcrlfx xxmaj fortunately , or unfortunately , that does not happen , but instead his speech brings about a civil war , just as he knew it would . \t▁ xcrlfx xxmaj marcus xxmaj antonius ' speech makes a very good example of the power of rhetoric , but it also illustrates the"
2,"but also people in the middle of their career , children , and retired people . xcrlfx xxmaj it is this group in general that is accused of not paying any attention at all to other people in traffic . xcrlfx xxmaj however , this group can build neither cycle stands nor cycle tracks and therefore they should not have to take this criticism . xcrlfx xxbos xxup no xxup to xxup","also people in the middle of their career , children , and retired people . xcrlfx xxmaj it is this group in general that is accused of not paying any attention at all to other people in traffic . xcrlfx xxmaj however , this group can build neither cycle stands nor cycle tracks and therefore they should not have to take this criticism . xcrlfx xxbos xxup no xxup to xxup the"
3,"person to be xxunk with . xcrlfx xxmaj in the book we learn as we go along that she is of a caring and nurturing nature and actually takes a liking to xxmaj gemmy , and feels comfortable with having him around , and his friends for that matter . xcrlfx xxmaj this only xxunk my view on her nurturing nature . xcrlfx xxmaj mrs xxmaj hutchence seem to long for a","to be xxunk with . xcrlfx xxmaj in the book we learn as we go along that she is of a caring and nurturing nature and actually takes a liking to xxmaj gemmy , and feels comfortable with having him around , and his friends for that matter . xcrlfx xxmaj this only xxunk my view on her nurturing nature . xcrlfx xxmaj mrs xxmaj hutchence seem to long for a big"
4,"xcrlfx xxmaj she choosed to bring xxmaj ben back home towards her family and relatives will , why ? xcrlfx xxmaj was it the ethical norms in the society ? xcrlfx xxmaj her quilt ? xcrlfx xxmaj nobody knows , and the book do n't give a clear answer to that questing , but it deals with such problem . \t▁ xcrlfx xxmaj back home xxmaj ben finally find his identity among","xxmaj she choosed to bring xxmaj ben back home towards her family and relatives will , why ? xcrlfx xxmaj was it the ethical norms in the society ? xcrlfx xxmaj her quilt ? xcrlfx xxmaj nobody knows , and the book do n't give a clear answer to that questing , but it deals with such problem . \t▁ xcrlfx xxmaj back home xxmaj ben finally find his identity among the"


In [42]:
defaults.device = dl.device

In [43]:
learn = language_model_learner(dl, AWD_LSTM,
                                metrics=[accuracy, Perplexity()],
                                path="./model/lm_pan14e/",
                                wd=0.1)

In [ ]:
def one_doc_embed(learn, doc):
    model = learn.model
    model = model.eval()
    
    pred_enc = model[0]
    doc_enc = model[0].encoder
    
    ids = dl.test_dl([doc]).items[0].to(dl.device)
    
    with torch.no_grad():
        pred_enc.reset()
        embed = doc_enc(ids[None]).detach().cpu()
        pred  = pred_enc(ids[None])[0][-1].detach().cpu()
        
        return {"e":embed, "p":pred}

In [44]:
model = learn.model
model.eval()
pred_enc = model[0]
pred_enc

AWD_LSTM(
  (encoder): Embedding(10392, 400, padding_idx=1)
  (encoder_dp): EmbeddingDropout(
    (emb): Embedding(10392, 400, padding_idx=1)
  )
  (rnns): ModuleList(
    (0): WeightDropout(
      (module): LSTM(400, 1152, batch_first=True)
    )
    (1): WeightDropout(
      (module): LSTM(1152, 1152, batch_first=True)
    )
    (2): WeightDropout(
      (module): LSTM(1152, 400, batch_first=True)
    )
  )
  (input_dp): RNNDropout()
  (hidden_dps): ModuleList(
    (0): RNNDropout()
    (1): RNNDropout()
    (2): RNNDropout()
  )
)

In [14]:
doc = df.iloc[i,2]
doc

' English, my English!  \t xcrlfx One of my first experiences with the English language was, when I at nine understood the contains of a conversation between my parents.  xcrlfx As they discussed what they thought to be a secret topic, hidden behind the language barrier, they were astonished when their little boy said "I didn\'t do it ". - Running away in triumph, finally having cracked their code.  xcrlfx After this incident my interest of listening to English, writing English and speaking the language only increased.   xcrlfx Now 12 years later it\'s interesting looking back at what I have learnt.   xcrlfx When I am having a conversation or when I am writing, it is obvious to me what are my strengths and weaknesses.  xcrlfx In the following paragraphs I will try to assess my overall abilities in the English language.   xcrlfx It\'s without a doubt that media has a great impact on young people in our society.  xcrlfx Radio, television and the Internet brings the world close to us.  xc

In [15]:
ids = dl.test_dl([doc]).items[0].to(dl.device)
ids

TensorText([   2,    8,   36,   12,   40,    8,   36,  230,    0,   10,    8,   60,
          15,   40,  135, 1110,   31,    9,    8,   36,   89,   37,   12,   43,
          20,   51, 1310,  897,    9, 1766,   15,   16,  996,  176,   40,  112,
          11,   10,    8,   25,   29,  825,   48,   29,  283,   13,   27,   16,
        2990, 1313,   12, 2324,  888,    9,   89, 6658,   12,   29,   71, 6382,
          43,   35,  232,  714,  266,   30,   20,  138,   61,   41,   21,   30,
          11,   34,    8, 1511,  346,   17, 6445,   12,  491,  226,    0,   35,
        5279,   11,   10,    8,  163,   28, 3046,   40,  549,   15,  218,   13,
           8,   36,   12,  157,    8,   36,   14,  175,    9,   89,   97,  519,
          11,   10,    8,  166, 1861,  133,  382,   21,   38,  513,  453,  289,
          51,   48,   20,   24, 1233,   11,   10,    8,   43,   20,  154,  226,
          16,  996,   47,   43,   20,  154,  157,   12,   21,   18,  588,   13,
         110,   48,   26,   40,  715

In [46]:
pred_enc = pred_enc.to(device=dl.device)

In [55]:
pred_enc.rnns[3].module._flat_weights

IndexError: index 3 is out of range

In [47]:
pred_enc( ids[None] )

RuntimeError: Input and hidden tensors are not at the same device, found input tensor at cuda:6 and hidden tensor at cpu